In [ ]:
%load_ext autoreload
%autoreload 2
import bokeh
from bokeh.models import ColumnDataSource, Label, Text, Span, HoverTool #, Range1d#, LabelSet
from bokeh.plotting import figure, output_file, show
import pandas as pd; pd.options.mode.chained_assignment = None # default warn => SettingWithCopyWarning
import numpy as np
from datetime import datetime
from pprint import pprint as pp
import warnings; warnings.simplefilter(action='ignore', category=UserWarning)
from IPython.display import HTML, display
from ipywidgets import AppLayout, Button, Layout, VBox, HBox, Label, HTML, interact, IntSlider
import ipywidgets as widgets
import arrow
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import scipy.fftpack

import time
import dmyplant2

#import dmyplant2
from dmyplant2 import (
    cred, MyPlant, Engine, 
    FSMOperator, filterFSM, FSM_splot, FSM_VLine, FSM_add_Notations, FSM_add_Alarms, FSM_add_Warnings, 
    bokeh_show, dbokeh_chart, add_dbokeh_vlines, get_cycle_data2, disp_result, disp_alarms, disp_warnings,
    cvset, load_data, get_cycle_data, get_cycle_data2, figures)
    
cred()
mp = MyPlant(3600)
Engine._list_cached_validations();
#mp._fetch_installed_base(); # refresh local installed fleet database

In [ ]:
def sfun(x):
    return all([ ("Forsa Hartmoor" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("EWB" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("Pforzheim" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("BMW Landshut 4.10" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
fleet.T;

In [ ]:
motor = fleet.iloc[0]
modes = ['undefined','OFF','MAN','AUTO']
success = [True,False]
#success = [True]
e=Engine.from_fleet(mp,motor)
#pp_from=e['Commissioning Date']
pp_from='2022-03-28 06:00'
pp_to='2022-03-28 08:14'
#pp_to=datetime.now()
fsm = FSMOperator(e, p_from=pp_from, p_to=pp_to)
#TODO:
# - move successtime to run1
motor['IB Site Name'] + ' ' + motor['Engine ID'], modes, success

In [ ]:
fsm.run0(enforce=True, silent=False, debug=False)
fsm.run1(silent=False, successtime=300, debug=False) # run Finite State Machine

In [ ]:
fsm.run2(silent = False)

In [ ]:
rdf = fsm.starts
rda = rdf[:].reset_index(drop='index')
rda = rda[(rda['mode'].isin(modes) & rda['success'].isin(success))].reset_index(drop='index')
rdb = rda
#rda[filterFSM.run2filter_content].round(2)
print(f"Starts: {rdf.shape[0]}, Successful: {rdf[rdf['success'] == True].shape[0]}, Failed: {rdf[rdf['success'] == False].shape[0]} => {rdf[rdf['success'] == True].shape[0]/rdf.shape[0]*100.0:3.1f}%")
pd.DataFrame.from_dict(e.dash, orient='index').T

In [ ]:
#rda = rda[rda['count_alarms'] > 0]
rda[filterFSM.run2filter_content].round(2).fillna('')

In [ ]:
rde = rda[(rda.starttime > fsm._e['Commissioning Date']) & (rda['success'])].copy()
rde['datetime'] = pd.to_datetime(rde['starttime'])
vec = ['startpreparation','speedup','idle','synchronize','loadramp','targetload','ramprate','cumstarttime','targetoperation','rampdown','coolrun','runout']
display(rde[vec].describe().round(2))
dfigsize = (18,12)
dset = [
    {'col':['cumstarttime'],'ylim':(-600,600), 'color':'darkblue'},
    {'col':['synchronize'],'ylim':(-20,400)},
    {'col':['startpreparation'],'ylim':(-1000,800)},
    {'col':['hochlauf'],'_ylim':(-100,200), 'color':'orange'},
    {'col':['loadramp'],'ylim':(-150,900), 'color':'red'},
    {'col':['ramprate'],'ylim':(-3,6)},
    #{'col':['no'],'ylim':(-100,2000),'color':'rgba(0,0,0,0.3)'},
    {'col':['targetload'],'ylim':(500,5000) },
    {'col':['runout'],'ylim':(0,100) },
    {'col':['count_warnings','count_alarms','no'],'ylim':(-1,200), 'color':['rgba(255,165,0,0.3)','rgba(255,0,0,0.3)','rgba(0,0,0,0.1)'] }
    ]
ftitle = f"{fsm._e}"
fig = dbokeh_chart(rde, dset, style='both', figsize=dfigsize ,title=ftitle);
bokeh_show(fig)

In [ ]:
def f(x=0):
    global vv; vv = x; startversuch = rdb.iloc[x]
    ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')} CumStart: {startversuch['cumstarttime']:0.1f}"
    display(HTML(ftitle));
    disp_result(startversuch)
interact(f, x = IntSlider(0, 0, rdb.shape[0]-1 , 1, layout=widgets.Layout(width='90%')));

In [ ]:
# PLotter
for f in figures:
    print(f, end=', ')
data = pd.DataFrame([])
for doplot in ['actors','lubrication']:
#for doplot in ['actors']:
    dmaxlength = None
    dminlength = None
    #dmaxlength = 1200
    #dminlength = 200000
    dset = figures[doplot]
    vset = cvset(mp,dset)
    startversuch = rdb.iloc[vv]; 
    try:
        data = get_cycle_data2(fsm, startversuch, max_length=dmaxlength, min_length=dminlength, cycletime=1, silent=True, p_data=cvset(mp,dset))
        data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
        fig = FSM_splot(fsm, startversuch, data, dset, figsize=(18,10))
        fig = FSM_add_Notations(fig, fsm, startversuch)
        disp_alarms(startversuch)
        disp_warnings(startversuch)
        fig = FSM_add_Alarms(fig, fsm, startversuch)
        fig = FSM_add_Warnings(fig, fsm, startversuch)
        bokeh_show(fig)
    except Exception as err:
        print(err)

In [ ]:

def xwhere(data,key,level):
    return data.iloc[data['datetime'][1:][np.array(data[key][1:]-level) * np.array(data[key][:-1]-level) < 0].index]

def xwhere2(data,key,level):
    pts = data['datetime'][1:][np.array(data[key][1:]-level) * np.array(data[key][:-1]-level) < 0]
    newlist =[x for p in list(pts.index) for x in [p-1,p]]
    return data.iloc[newlist]

def xwhere_dir(data,key,level,direction):
    data['diff'] = pd.Series(np.diff(data[key].values))
    pts = data['datetime'][1:][np.array(data[key][1:]-level) * np.array(data[key][:-1]-level) < 0]
    if direction == 'up':
        erg = data.iloc[pts.index][data['diff'] > 0]
    elif direction == 'down': 
        erg = data.iloc[pts.index][data['diff'] < 0]
    else:
        erg = data.iloc[pts.index]
    return erg

xwhere_dir(data,'Various_Values_SpeedAct',50,'up')

In [ ]:
op0 = xwhere(data,'Hyd_PressOil',1.1)
op1 = xwhere(data,'Hyd_PressOil',0.1)
op2 = xwhere(data,'Hyd_PressOil',0.5)
prelube_degasing_time = (op0.iloc[0]['time'] - op1.iloc[0]['time']) / 1000.0
post_lube_time = (op1.iloc[1]['time'] - op2.iloc[1]['time']) / (1000.0 * 60) 
prelube_degasing_time, 'sec' , post_lube_time, 'min'

In [ ]:
xwhere(data,'Hyd_PressOil',0.5)

In [ ]:
xwhere(data,'Hyd_PressOil',0.1)

In [ ]:
xwhere(data,'Various_Values_SpeedAct',1503)

In [ ]:
def cdata(startversuch, data, phases):
    ts = int(startversuch['startstoptiming'][phases[0]][0]['start'].timestamp() * 1000)
    te = int(startversuch['startstoptiming'][phases[-1]][0]['end'].timestamp() * 1000)
    return data[(data.time > ts) & (data.time < te)].reset_index(drop=True)

def extract_data(startversuch, data, dataItemName, phases):
    ts = int(startversuch['startstoptiming'][phases[0]][0]['start'].timestamp() * 1000)
    te = int(startversuch['startstoptiming'][phases[-1]][0]['end'].timestamp() * 1000)
    return data[['datetime',dataItemName]][(data.time > ts) & (data.time < te)].reset_index(drop=True)

def edges_in_phase2(startversuch, data, dataItemName, phases, direction, ax=None):
    c = extract_data(startversuch, data, dataItemName, phases)
    c['_diff'] = pd.Series(np.diff(c[dataItemName].values))
    pts = xwhere_dir(c, '_diff', c['_diff'].max() * 0.1, direction)
    if ax is not None:
        ax.plot(c['datetime'], c[dataItemName])
        for i,p in pts.iterrows():
            ax.axvline(p['datetime'],linestyle='--', c='k', alpha=0.8, linewidth=1)
        ax.grid()        
    return pts

def FSM_pre_phases(startversuch, data, dataItemName, phases, direction, fig=None, y_pos=2, color='rgba(0,0,0,0.8)', line='solid', alpha=1.0, perc=0.11):
    c = extract_data(startversuch, data, dataItemName, phases)
    c['_diff'] = pd.Series(np.diff(c[dataItemName].values))
    if direction == 'up':
        tresh_hold = c['_diff'].max()
    elif direction == 'down':
        tresh_hold = c['_diff'].min()
    pts = xwhere_dir(c, '_diff', tresh_hold * perc, direction)
    if fig is not None:
        for i,p in pts.iterrows():
            FSM_VLine(fig, txt=f"{p['datetime'].strftime('%Y-%m-%d %H:%M:%S')} {dataItemName}", x_pos=p['datetime'], y_pos=y_pos, color=color, line=line, alpha=alpha)
            #ax.axvline(p['datetime'],linestyle='--', c='k', alpha=0.8, linewidth=1)
    return fig

def FSM_start(startversuch, data, dataItemName, phases, fig=None, y_pos=2, color='rgba(0,0,0,0.8)', line='solid', alpha=1.0, perc=0.11):
    c = extract_data(startversuch, data, dataItemName, phases)
    pts = xwhere_dir(c, dataItemName, c[dataItemName].max() * perc, 'up')
    if fig is not None:
        for i,p in pts.iterrows():
            FSM_VLine(fig, txt=f"{p['datetime'].strftime('%Y-%m-%d %H:%M:%S')} {dataItemName}", x_pos=p['datetime'], y_pos=y_pos, color=color, line=line, alpha=alpha)
            #ax.axvline(p['datetime'],linestyle='--', c='k', alpha=0.8, linewidth=1)
    return fig

d0 = cdata(startversuch, data, ['startpreparation','starter','speedup','idle','synchronize','targetoperation']).iloc[:400,:]
d = cdata(startversuch, data, ['startpreparation','starter','speedup'])

#fig = plt.figure(figsize=(12,8))
#ax = fig.add_subplot()
#ax.plot('datetime','Various_Values_SpeedAct', data=d)
#play(startversuch, data, 'Hyd_PressOil',['startpreparation','starter','speedup'],'up', ax)

In [ ]:
def smooth(y,box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

def smooth_fft(y, cut, dt):
    w = scipy.fftpack.rfft(y)
    f = scipy.fftpack.rfftfreq(len(y),dt)
    spectrum = w**2
    cutoff_idx = spectrum < (spectrum.max()/cut)
    w2 = w.copy()
    w2[cutoff_idx] = 0
    return scipy.fftpack.irfft(w2)

In [ ]:

#d0['gHyd_PressOilDif'] = pd.Series(savitzky_golay(np.array(d0['Hyd_PressOilDif']), 51, 3)) # window size 51, polynomial order 3
#d0['gHyd_PressOilDif'] = pd.Series(savitzky_golay(np.array(d0['Hyd_PressOilDif']), 29, 3)) # window size 51, polynomial order 3
d0['gHyd_PressOilDif'] = pd.Series(savgol_filter(np.array(d0['Hyd_PressOilDif']), 29, 3)) # window size 51, polynomial order 3
d0['hHyd_PressOilDif'] = pd.Series(smooth(np.array(d0['Hyd_PressOilDif']), 13))
d0['iHyd_PressOilDif'] = pd.Series(smooth_fft(np.array(d0['Hyd_PressOilDif']), 5000, 1))
fig = plt.figure(figsize=(16,12))
ax = fig.add_subplot()
ax.plot(d0['datetime'],d0['Hyd_PressOilDif'], 'b.', alpha=0.6)
ax.plot(d0['datetime'],d0['hHyd_PressOilDif'], 'g-', lw=4)
ax.plot(d0['datetime'],d0['gHyd_PressOilDif'], 'r-', lw=2)
ax.plot(d0['datetime'],d0['iHyd_PressOilDif'], 'k-', lw=2)
ax.grid()


In [ ]:
#np.array(d0['Hyd_PressOilDif'])
data

In [ ]:
dset = figures['lubrication']
vset = cvset(mp,dset)
fig = FSM_splot(fsm, startversuch, d0, dset, figsize=(18,10))
fig = FSM_add_Notations(fig, fsm, startversuch)
fig = FSM_add_Alarms(fig, fsm, startversuch)
fig = FSM_add_Warnings(fig, fsm, startversuch)
#fig = FSM_pre_phases(startversuch, data, 'Hyd_PressOil',['startpreparation','starter'],'up', fig, y_pos=50, color='brown')
fig = FSM_start(startversuch, data, 'Hyd_PressOil',['startpreparation'], fig, y_pos=50, color='brown',perc=0.05)
fig = FSM_start(startversuch, data, 'Various_Values_SpeedAct',['startpreparation','starter'], fig, y_pos=50, color='blue',perc=0.05)
#fig = FSM_pre_phases(startversuch, data, 'Various_Values_SpeedAct',['startpreparation','starter','speedup'],'up', fig, y_pos=250, color='blue')
bokeh_show(fig)

In [ ]:
# Algorithm visualization
dset2 = [{'col':['Power_PowerAct','helpline'], 'ylim':(-1000,12000), 'color':['red','rgba(0,128,0,0.2)'], 'unit':'kW'},
         {'col':['power_diff','power_diff_help'], '_ylim':(0,5000), 'color':['rgba(255,0,0,0.2)','rgba(0,128,0,0.2)'], 'unit':'kW/s'}]
ratedload = fsm._e['Power_PowerNominal']
interestingcycles = [vv] # Bautzen M01 T902
periodfactor = 3 # times calculated period length
helplinefactor = 0.8 # overlay slope to -fac * end value 
for i in interestingcycles: 
    #startversuch=fsm.results['starts'][i]
    startversuch=rdb.iloc[i]
    disp_result(startversuch)
    sno = startversuch['no']
    data, xmax, ymax, duration, ramprate = dmyplant2.loadramp_edge_detect(fsm,startversuch, periodfactor=periodfactor, helplinefactor=helplinefactor)
    if not data.empty:
        print(f"Start: {startversuch['no']:3d} xmax: {xmax}, ymax: {ymax:6.0f}, duration: {duration:5.1f}, ramprate: {ramprate / ratedload * 100.0:4.2f} %/s")
        data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
        #data['power_diff_help'] = pd.Series(np.gradient(data['helpline']))
        fig = FSM_splot(fsm, startversuch, data, dset2, figsize=(16,8))
        fig.add_layout(Span(location=0.0,dimension='width',x_range_name='default', y_range_name='0',line_color='black', line_dash='solid', line_alpha=0.4)) 
        fig.add_layout(Span(location=ymax,dimension='width',x_range_name='default', y_range_name='0',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2)) 
        fig.add_layout(Span(location=xmax,dimension='height',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2)) 
        fig = FSM_add_Notations(fig, fsm, startversuch)
        fig = FSM_add_Alarms(fig, fsm, startversuch)
        fig = FSM_add_Warnings(fig, fsm, startversuch)
        bokeh_show(fig)
    else:
        print(f"Start: {startversuch['no']:3d} no data, no improvement possible.")

In [ ]:
fsm.results['serviceselectortiming'];

In [ ]:
for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=True)):
    print(f"{i:3} {v}")

In [ ]:
for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=False)):
    print(f"{i:3} {v}")

In [ ]:
rdb = rda
vec = ['startpreparation','speedup','idle','synchronize','loadramp','targetload','ramprate','cumstarttime','targetoperation','rampdown','coolrun','runout']
display(_=rdb[vec].hist(bins=30,figsize=(20,20)))
#ax_list[0][2].set_xlim((0,10))
#display(rdb[vec].hist(bins=30,figsize=(20,20)))
display(rdb[vec].describe().round(2))

In [ ]:
dset2 = [{'col':['Power_PowerAct','helpline'], 'ylim':(-1000,12000), 'color':['red','rgba(0,128,0,0.2)'], 'unit':'kW'},
         {'col':['power_diff','power_diff_help'], '_ylim':(0,5000), 'color':['rgba(255,0,0,0.2)','rgba(0,128,0,0.2)'], 'unit':'kW/s'}]
ratedload = fsm._e['Power_PowerNominal']
periodfactor = 3 # times calculated period length
helplinefactor = 0.8 # overlay slope to -fac * end value 
t0 = time.time()
for i, startversuch in rdb[-1:].iterrows() : 
#for i, startversuch in tqdm(rdb.iterrows(), total=rdb.shape[0], ncols=80, mininterval=1, unit=' starts', desc="FSM Run2"):
    data, xmax, ymax, duration, ramprate = dmyplant2.loadramp_edge_detect(fsm,startversuch, periodfactor, helplinefactor)
    if not data.empty:
        print(f"Start: {startversuch['no']:3d} xmax: {xmax}, ymax: {ymax:6.0f}, duration: {duration:5.1f}, ramprate: {ramprate / ratedload * 100.0:4.2f} %/s")
        data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
        #data['power_diff_help'] = pd.Series(np.gradient(data['helpline']))
        fig = FSM_splot(fsm, startversuch, data, dset2, figsize=(14,6))
        fig.add_layout(Span(location=0.0,dimension='width',x_range_name='default', y_range_name='0',line_color='black', line_dash='solid', line_alpha=0.4)) 
        fig.add_layout(Span(location=ymax,dimension='width',x_range_name='default', y_range_name='0',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2)) 
        fig.add_layout(Span(location=xmax,dimension='height',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2)) 
        fig = FSM_add_Notations(fig, fsm, startversuch)
        #fig = FSM_add_Alarms(fig, fsm, startversuch)
        #fig = FSM_add_Warnings(fig, fsm, startversuch)
        bokeh_show(fig)
    else:
        print(f"Start: {startversuch['no']:3d} no data, no improvement possible.")
t1 = time.time()
print(f"{(t1-t0):4.1f} sec")    

In [ ]:
mfn = e._fname + '_messages.txt'
fsm.save_messages(mfn)
print(mfn)